<a href="https://colab.research.google.com/github/Rakesh-kumar-s/Rakesh-kumar-s/blob/main/Addressing_Domestic_Violence_in_India.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pandas numpy matplotlib seaborn geopandas folium plotly

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from google.colab import files

In [8]:
# Upload NFHS-5 data (download from https://dhsprogram.com)
import io
uploaded = files.upload()
# Assuming the uploaded file is named 'NFHS_DV_India.csv', access its content from the uploaded dictionary
dv_data = pd.read_csv(io.StringIO(uploaded['NFHS_DV_India.csv'].decode('utf-8')))

# Sample police-recorded cases (NCRB data)
uploaded = files.upload()
# Assuming the uploaded file is named 'NCRB_cases.csv', access its content from the uploaded dictionary
ncrb_data = pd.read_csv(io.StringIO(uploaded['NCRB_cases.csv'].decode('utf-8')))

Saving datafile.csv to datafile (2).csv


KeyError: 'NFHS_DV_India.csv'

In [ ]:
# Clean and prepare data
statewise = dv_data.groupby('State')['Prevalence'].mean().reset_index()
statewise = statewise.sort_values('Prevalence', ascending=False)

# Interactive choropleth map
fig = px.choropleth(
    statewise,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    locations='State',
    featureidkey='properties.ST_NM',
    color='Prevalence',
    color_continuous_scale='Reds',
    title='Domestic Violence Prevalence by State (NFHS-5)'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
# Plot yearly trends
plt.figure(figsize=(12, 6))
sns.lineplot(data=ncrb_data, x='Year', y='Cases_Reported', hue='State',
             style='State', markers=True, dashes=False)
plt.title('Reported Domestic Violence Cases (NCRB Data)')
plt.ylabel('Cases Reported (Thousands)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Merge NFHS and NCRB data
merged_data = pd.merge(
    dv_data,
    ncrb_data,
    on=['State', 'Year'],
    how='inner'
)

# Calculate risk factors
merged_data['Risk_Score'] = (
    0.4 * merged_data['Prevalence'] +
    0.3 * merged_data['Alcohol_Consumption'] +
    0.2 * merged_data['Female_Literacy'] +
    0.1 * merged_data['Poverty_Rate']
)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Prepare features
X = merged_data[['Alcohol_Consumption', 'Female_Literacy', 'Poverty_Rate']]
y = (merged_data['Prevalence'] > 30).astype(int)  # Binary classification

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate
print(f"Model Accuracy: {model.score(X_test, y_test):.2f}")

In [ ]:
def calculate_resources(state, population):
    prevalence = dv_data[dv_data['State']==state]['Prevalence'].values[0]
    shelters_needed = round((population * prevalence/100) / 5000)
    counselors = round((population * prevalence/100) / 10000)

    print(f"Recommended for {state}:")
    print(f"- Shelter Homes: {shelters_needed}")
    print(f"- Counselors: {counselors}")
    print(f"- Police Help Desks: {shelters_needed*2}")

calculate_resources('Karnataka', 67500000)

In [ ]:
!pip install ipywidgets
from ipywidgets import interact, Dropdown

states = dv_data['State'].unique()
@interact(State=Dropdown(options=states))
def show_state_stats(State):
    state_data = dv_data[dv_data['State']==State]
    display(state_data.style.highlight_max(color='lightcoral'))

In [ ]:
# Generate localized awareness messages
def generate_message(state, language):
    messages = {
        'Hindi': f"{state} में घरेलू हिंसा रिपोर्ट करें: 181 (महिला हेल्पलाइन)",
        'English': f"Report domestic violence in {state}: Call 181 (Women Helpline)",
        'Kannada': f"{state}ನಲ್ಲಿ ಗೃಹ ಹಿಂಸೆಯನ್ನು ವರದಿ ಮಾಡಿ: 181 (ಮಹಿಳಾ ಸಹಾಯ ಲೈನ್)"
    }
    return messages.get(language, "Language not supported")

print(generate_message('Karnataka', 'Kannada'))